In [1]:
from model import GNNGlobal, GNNTopK, GNNSAG, GNNDiffPool, GNNGlobalAttention, GNNSet2Set, GNNDMoN, GNNECPool, GNNMinCut
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
import torch
import torch.nn.functional as F
import time
import pandas as pd
import gc

In [2]:
def train(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)  # Forward pass
        loss = F.cross_entropy(out, data.y)   # Compute cross-entropy loss between predictions and true labels
        loss.backward()     # Backpropagation
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)  # Return average loss per graph             # Return average loss per graph

"""Evaluate model accuracy on the given DataLoader."""
@torch.no_grad()   # Disables autograd tracking
def test(model, loader, device):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        pred = model(data.x, data.edge_index, data.batch).argmax(dim=1)   # Predict class by selecting highest logit
        correct += (pred == data.y).sum().item()
    
    return correct / len(loader.dataset)


In [ ]:
results = []
for name in ['MUTAG', 'PROTEINS', 'ENZYMES']:
    dataset = TUDataset(root='data', name=name)  # Load the graph classification dataset
    dataset = dataset.shuffle()  # Shuffle the dataset

    # Split the dataset into training and testing subsets
    split = int(0.8 * len(dataset))
    train_ds, test_ds = dataset[:split], dataset[split:]
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=32)

    in_feats = dataset.num_features
    hidden_dim = 64
    num_classes = dataset.num_classes
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    print(f"Dataset: {name}, Number of classes: {num_classes}, Number of features: {in_feats}")

    # Grid search over k values
    for r in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:  # Define the range of k values to test
        print(f"For {name} dataset with ratio ={r}:")
        model = GNNECPool(in_feats, hidden_dim, num_classes, ratio = r).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

        if device.type == 'cuda':
            torch.cuda.reset_peak_memory_stats()

        start_time = time.time()
        for epoch in range(1, 101):
            loss = train(model, train_loader, optimizer, device)
        total_time = time.time() - start_time
        time_per_epoch = total_time / 100

        acc = test(model, test_loader, device)

        if device.type == 'cuda':
            peak_mem = torch.cuda.max_memory_allocated(device) / (1024 ** 2)  # Convert to MB
        else:
            peak_mem = None

        print(f"k={r}, test acc: {acc}, time/epoch: {time_per_epoch:.4f}s")
        # torch.save(model.state_dict(), f"trained_models/{name}_dmon_k{k}_model.pth")
        results.append({
            'k': r,
            'dataset': name,
            's/epoch': round(time_per_epoch, 4),
            'memory_MB': round(peak_mem, 2) if peak_mem is not None else 'N/A',
            'accuracy': round(acc, 4)
        })

        # Cleanup to avoid CUDA OOM across variants
        del model
        gc.collect()
        if device.type == 'cuda':
            torch.cuda.empty_cache()

# Convert to DataFrame
results_df = pd.DataFrame(results)
# results_df.to_csv(f'gnn_pooling_benchmark_k_grid_search.csv', index=False)
print(results_df)


Using device: cpu
Dataset: MUTAG, Number of classes: 2, Number of features: 7
For MUTAG dataset with ratio =0.1:


/home/utkarsh/miniconda3/envs/aiml/lib/python3.7/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


k=0.1, test acc: 0.8157894736842105, time/epoch: 0.0591s
For MUTAG dataset with ratio =0.2:
k=0.2, test acc: 0.8157894736842105, time/epoch: 0.0523s
For MUTAG dataset with ratio =0.3:
k=0.3, test acc: 0.8421052631578947, time/epoch: 0.0512s
For MUTAG dataset with ratio =0.4:
k=0.4, test acc: 0.8421052631578947, time/epoch: 0.0569s
For MUTAG dataset with ratio =0.5:
k=0.5, test acc: 0.868421052631579, time/epoch: 0.0485s
For MUTAG dataset with ratio =0.6:
k=0.6, test acc: 0.7894736842105263, time/epoch: 0.0524s
For MUTAG dataset with ratio =0.7:
k=0.7, test acc: 0.8421052631578947, time/epoch: 0.0514s
For MUTAG dataset with ratio =0.8:
k=0.8, test acc: 0.868421052631579, time/epoch: 0.0510s
For MUTAG dataset with ratio =0.9:
k=0.9, test acc: 0.8157894736842105, time/epoch: 0.0599s
Using device: cpu
Dataset: PROTEINS, Number of classes: 2, Number of features: 3
For PROTEINS dataset with ratio =0.1:
k=0.1, test acc: 0.7354260089686099, time/epoch: 0.7300s
For PROTEINS dataset with ratio =

In [ ]:
results = []
for name in ['MUTAG', 'PROTEINS', 'ENZYMES']:
    dataset = TUDataset(root='data', name=name)  # Load the graph classification dataset
    dataset = dataset.shuffle()  # Shuffle the dataset

    # Split the dataset into training and testing subsets
    split = int(0.8 * len(dataset))
    train_ds, test_ds = dataset[:split], dataset[split:]
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=32)

    in_feats = dataset.num_features
    hidden_dim = 64
    num_classes = dataset.num_classes
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    print(f"Dataset: {name}, Number of classes: {num_classes}, Number of features: {in_feats}")

    # Grid search over k values
    for r in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:  # Define the range of k values to test
        print(f"For {name} dataset with ratio ={r}:")
        model = GNNSAG(in_feats, hidden_dim, num_classes, ratio = r).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

        if device.type == 'cuda':
            torch.cuda.reset_peak_memory_stats()

        start_time = time.time()
        for epoch in range(1, 101):
            loss = train(model, train_loader, optimizer, device)
        total_time = time.time() - start_time
        time_per_epoch = total_time / 100

        acc = test(model, test_loader, device)

        if device.type == 'cuda':
            peak_mem = torch.cuda.max_memory_allocated(device) / (1024 ** 2)  # Convert to MB
        else:
            peak_mem = None

        print(f"k={r}, test acc: {acc}, time/epoch: {time_per_epoch:.4f}s")
        # torch.save(model.state_dict(), f"trained_models/{name}_dmon_k{k}_model.pth")
        results.append({
            'k': r,
            'dataset': name,
            's/epoch': round(time_per_epoch, 4),
            'memory_MB': round(peak_mem, 2) if peak_mem is not None else 'N/A',
            'accuracy': round(acc, 4)
        })

        # Cleanup to avoid CUDA OOM across variants
        del model
        gc.collect()
        if device.type == 'cuda':
            torch.cuda.empty_cache()

# Convert to DataFrame
results_df = pd.DataFrame(results)
# results_df.to_csv(f'gnn_pooling_benchmark_k_grid_search.csv', index=False)
print(results_df)


Using device: cpu
Dataset: MUTAG, Number of classes: 2, Number of features: 7
For MUTAG dataset with ratio =0.1:
k=0.1, test acc: 0.7368421052631579, time/epoch: 0.0290s
For MUTAG dataset with ratio =0.2:
k=0.2, test acc: 0.7368421052631579, time/epoch: 0.0281s
For MUTAG dataset with ratio =0.3:
k=0.3, test acc: 0.7631578947368421, time/epoch: 0.0350s
For MUTAG dataset with ratio =0.4:
k=0.4, test acc: 0.7368421052631579, time/epoch: 0.0256s
For MUTAG dataset with ratio =0.5:
k=0.5, test acc: 0.7631578947368421, time/epoch: 0.0293s
For MUTAG dataset with ratio =0.6:
k=0.6, test acc: 0.7368421052631579, time/epoch: 0.0371s
For MUTAG dataset with ratio =0.7:
k=0.7, test acc: 0.7105263157894737, time/epoch: 0.0374s
For MUTAG dataset with ratio =0.8:
k=0.8, test acc: 0.7894736842105263, time/epoch: 0.0324s
For MUTAG dataset with ratio =0.9:
k=0.9, test acc: 0.7894736842105263, time/epoch: 0.0383s
Using device: cpu
Dataset: PROTEINS, Number of classes: 2, Number of features: 3
For PROTEINS 

In [ ]:
results = []
for name in ['ENZYMES']:
    
    dataset = TUDataset(root='data', name=name)    # Load the graph classification dataset into a PyTorch Geometric TUDataset object
    dataset = dataset.shuffle()  
    split = int(0.8 * len(dataset))
    train_ds, test_ds = dataset[:split], dataset[split:] 
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    test_loader  = DataLoader(test_ds,  batch_size=32)
    in_feats    = dataset.num_features   # Number of node features per graph
    hidden_dim  = 64                     # Hidden dimension size for internal GCN layers
    num_classes = dataset.num_classes    # Number of target classes for graph-level classification
    device      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Compute device
    print(f"Using device: {device}")
    # print(f"Dataset: {name}, Number of classes: {num_classes}, Number of features: {in_feats}")
    variants = {
        'dmon':          GNNDMoN(in_feats, hidden_dim, num_classes, k=11, dropout=0.2),
        'ecpool':        GNNECPool(in_feats, hidden_dim, num_classes, ratio=0.1),
        # 'mincut':        GNNMinCut(in_feats, hidden_dim, num_classes, k=10, temp = 1.0),
        'set2set':     GNNSet2Set(in_feats, hidden_dim, num_classes, 3),
        'topk':        GNNTopK(in_feats, hidden_dim, num_classes, 0.4),
        'sag':         GNNSAG(in_feats, hidden_dim, num_classes, ratio=0.8),
        'diff':        GNNDiffPool(in_feats, assign_dim=hidden_dim, k=11, num_classes=num_classes)
    }

    print(f"For {name} dataset:")
    for pool, model in variants.items():
        model = model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

        if device.type == 'cuda':
            torch.cuda.reset_peak_memory_stats()

        start_time = time.time()
        for epoch in range(1, 101):
            loss = train(model, train_loader, optimizer, device)
        total_time = time.time() - start_time
        time_per_epoch = total_time / 100

        acc = test(model, test_loader, device)

        if device.type == 'cuda':
            peak_mem = torch.cuda.max_memory_allocated(device) / (1024 ** 2)  # Convert to MB
        else:
            peak_mem = None  # Could use psutil here for CPU if needed

        print(f"{pool} test acc: {acc:.4f}, time/epoch: {time_per_epoch:.4f}s")
        torch.save(model.state_dict(), f"optimals/{name}_{pool}_model.pth")
        results.append({
            'pool': pool,
            'dataset': name,
            's/epoch': round(time_per_epoch, 4),
            'memory_MB': round(peak_mem, 2) if peak_mem is not None else 'N/A',
            'accuracy': round(acc, 4)
        })

        # Cleanup to avoid CUDA OOM across variants
        del model
        gc.collect()
        if device.type == 'cuda':
            torch.cuda.empty_cache()

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv(f'gnn_pooling_benchmark_opt.csv', index=False)
print(results_df)

Using device: cpu
For ENZYMES dataset:
dmon test acc: 0.2583, time/epoch: 0.1003s
ecpool test acc: 0.3167, time/epoch: 0.3749s
set2set test acc: 0.4667, time/epoch: 0.2170s
topk test acc: 0.2583, time/epoch: 0.0773s
sag test acc: 0.3000, time/epoch: 0.0964s
diff test acc: 0.4833, time/epoch: 0.1028s
      pool  dataset  s/epoch memory_MB  accuracy
0     dmon  ENZYMES   0.1003       N/A    0.2583
1   ecpool  ENZYMES   0.3749       N/A    0.3167
2  set2set  ENZYMES   0.2170       N/A    0.4667
3     topk  ENZYMES   0.0773       N/A    0.2583
4      sag  ENZYMES   0.0964       N/A    0.3000
5     diff  ENZYMES   0.1028       N/A    0.4833


In [ ]:
results = []
for name in ['MUTAG']:
    
    dataset = TUDataset(root='data', name=name)    # Load the graph classification dataset into a PyTorch Geometric TUDataset object
    dataset = dataset.shuffle()  
    split = int(0.8 * len(dataset))
    train_ds, test_ds = dataset[:split], dataset[split:] 
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    test_loader  = DataLoader(test_ds,  batch_size=32)
    in_feats    = dataset.num_features   # Number of node features per graph
    hidden_dim  = 64                     # Hidden dimension size for internal GCN layers
    num_classes = dataset.num_classes    # Number of target classes for graph-level classification
    device      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Compute device
    print(f"Using device: {device}")
    # print(f"Dataset: {name}, Number of classes: {num_classes}, Number of features: {in_feats}")
    variants = {
        'dmon':          GNNDMoN(in_feats, hidden_dim, num_classes, k=16, dropout=0.2),
        'ecpool':        GNNECPool(in_feats, hidden_dim, num_classes, ratio=0.8),
        # 'mincut':        GNNMinCut(in_feats, hidden_dim, num_classes, k=10, temp = 1.0),
        'set2set':     GNNSet2Set(in_feats, hidden_dim, num_classes, 3),
        'topk':        GNNTopK(in_feats, hidden_dim, num_classes, 0.3),
        'sag':         GNNSAG(in_feats, hidden_dim, num_classes, ratio=0.9),
        'diff':        GNNDiffPool(in_feats, assign_dim=hidden_dim, k=19, num_classes=num_classes)
    }

    print(f"For {name} dataset:")
    for pool, model in variants.items():
        model = model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

        if device.type == 'cuda':
            torch.cuda.reset_peak_memory_stats()

        start_time = time.time()
        for epoch in range(1, 101):
            loss = train(model, train_loader, optimizer, device)
        total_time = time.time() - start_time
        time_per_epoch = total_time / 100

        acc = test(model, test_loader, device)

        if device.type == 'cuda':
            peak_mem = torch.cuda.max_memory_allocated(device) / (1024 ** 2)  # Convert to MB
        else:
            peak_mem = None  # Could use psutil here for CPU if needed

        print(f"{pool} test acc: {acc:.4f}, time/epoch: {time_per_epoch:.4f}s")
        torch.save(model.state_dict(), f"optimals/{name}_{pool}_model.pth")
        results.append({
            'pool': pool,
            'dataset': name,
            's/epoch': round(time_per_epoch, 4),
            'memory_MB': round(peak_mem, 2) if peak_mem is not None else 'N/A',
            'accuracy': round(acc, 4)
        })

        # Cleanup to avoid CUDA OOM across variants
        del model
        gc.collect()
        if device.type == 'cuda':
            torch.cuda.empty_cache()

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv(f'gnn_pooling_benchmark_opt.csv', index=False)
print(results_df)

/home/utkarsh/miniconda3/envs/aiml/lib/python3.7/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Using device: cpu
For MUTAG dataset:
dmon test acc: 0.9211, time/epoch: 0.0380s
ecpool test acc: 0.8684, time/epoch: 0.0538s
set2set test acc: 0.8421, time/epoch: 0.0497s
topk test acc: 0.8684, time/epoch: 0.0178s
sag test acc: 0.8684, time/epoch: 0.0307s
diff test acc: 0.8947, time/epoch: 0.0386s
      pool dataset  s/epoch memory_MB  accuracy
0     dmon   MUTAG   0.0380       N/A    0.9211
1   ecpool   MUTAG   0.0538       N/A    0.8684
2  set2set   MUTAG   0.0497       N/A    0.8421
3     topk   MUTAG   0.0178       N/A    0.8684
4      sag   MUTAG   0.0307       N/A    0.8684
5     diff   MUTAG   0.0386       N/A    0.8947
